In [1]:
#from sqlalchemy import create_engine
# Set Up Environment
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize

In [15]:
# Database credentials
postgres_user = 'postgres'
postgres_pw = 'postgres'
postgres_host = 'localhost'
postgres_port = '5432'
postgres_db = 'bev_rec'

# Use the credentials to start a connection
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

ModuleNotFoundError: No module named 'sqlalchemy'

In [2]:
# Initial sql query, stored in sql variable
sql = '''
  SELECT *
  FROM public.test;
'''
results = engine.execute(sql)
engine.dispose()
rows = results.fetchall()

# print some results just to see what we got
for row in rows:
  print(row)

(1, 'Michael')


In [9]:
# List of parameters to search DB
search_parameters = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

url = 'https://www.thecocktaildb.com/api/json/v1/1/search.php?f='
json_list = []

for i in search_parameters:
    r = requests.get(url+str(i))
    # This presumes your JSON response is a dict, if the response is a list, use extend instead of append
    json_list.append(r.json())
    
df = pd.json_normalize(json_list, 'drinks')

In [10]:
# Filter out 
df_new = df[['idDrink', 'strDrink', 'strCategory', 'strIBA', 'strAlcoholic', 
         'strGlass', 'strInstructions', 'strDrinkThumb', 'strIngredient1', 
         'strIngredient2', 'strIngredient3', 'strIngredient4', 'strIngredient5', 
         'strIngredient6', 'strIngredient7', 'strIngredient8', 'strIngredient9', 
         'strIngredient10', 'strMeasure1', 'strMeasure2', 'strMeasure3', 'strMeasure4', 
         'strMeasure5', 'strMeasure6', 'strMeasure7', 'strMeasure8', 'strMeasure9', 'strMeasure10']]

# Create all ingredients column
df_new['all_ingredients'] = df_new[df_new.columns[8:17]].apply(
    lambda x: ', '.join(x.dropna().astype(str)), axis=1)

# Create soup column
df_new['soup'] = df_new[['all_ingredients', 'strInstructions']].apply(lambda x: ','.join(x), axis=1)

# Save new dataframe as csv
df_new.to_csv(r'../static/cocktaildb.csv', index=True, header=True)

# Get list of drink names
newlist = pd.Series([x for x in df['strDrink'].str.replace("'", "")])

df_new.head()

<ipython-input-10-cd70d8d16440>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['all_ingredients'] = df_new[df_new.columns[8:17]].apply(
<ipython-input-10-cd70d8d16440>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['soup'] = df_new[['all_ingredients', 'strInstructions']].apply(lambda x: ','.join(x), axis=1)


,idDrink,strDrink,strCategory,strIBA,strAlcoholic,strGlass,strInstructions,strDrinkThumb,strIngredient1,strIngredient2,...,strMeasure3,strMeasure4,strMeasure5,strMeasure6,strMeasure7,strMeasure8,strMeasure9,strMeasure10,all_ingredients,soup
0,15346,155 Belmont,Cocktail,None,Alcoholic,White wine glass,Blend with ice. Serve in a wine glass. Garnish...,https://www.thecocktaildb.com/images/media/dri...,Dark rum,Light rum,...,1 shot,1 shot,None,None,None,None,None,None,"Dark rum, Light rum, Vodka, Orange juice","Dark rum, Light rum, Vodka, Orange juice,Blend..."
1,15395,1-900-FUK-MEUP,Shot,None,Alcoholic,Old-fashioned glass,Shake ingredients in a mixing tin filled with ...,https://www.thecocktaildb.com/images/media/dri...,Absolut Kurant,Grand Marnier,...,1/4 oz,1/4 oz,1/4 oz,1/4 oz,1/2 oz,1/4 oz,None,None,"Absolut Kurant, Grand Marnier, Chambord raspbe...","Absolut Kurant, Grand Marnier, Chambord raspbe..."
2,15423,110 in the shade,Beer,None,Alcoholic,Beer Glass,Drop shooter in glass. Fill with beer,https://www.thecocktaildb.com/images/media/dri...,Lager,Tequila,...,None,None,None,None,None,None,None,None,"Lager, Tequila","Lager, Tequila,Drop shooter in glass. Fill wit..."
3,14588,151 Florida Bushwacker,Milk / Float / Shake,None,Alcoholic,Beer mug,Combine all ingredients. Blend until smooth. G...,https://www.thecocktaildb.com/images/media/dri...,Malibu rum,Light rum,...,1/2 oz Bacardi,1 oz,1 oz,3 oz,1 oz,1 cup,None,None,"Malibu rum, Light rum, 151 proof rum, Dark Cre...","Malibu rum, Light rum, 151 proof rum, Dark Cre..."
4,15288,252,Shot,None,Alcoholic,Shot glass,"Add both ingredients to shot glass, shoot, and...",https://www.thecocktaildb.com/images/media/dri...,151 proof rum,Wild Turkey,...,None,None,None,None,None,None,None,None,"151 proof rum, Wild Turkey","151 proof rum, Wild Turkey,Add both ingredient..."


In [20]:
# Get list of drink names
newlist = pd.Series([x for x in df['strDrink'].str.replace("'", "")])


In [21]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df_new['all_ingredients'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

#Construct a reverse map of indices and movie titles
indices = pd.Series(df_new.index, index=df_new['strDrink']).drop_duplicates()

# Function that takes in drink name as input and outputs most similar drinks
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the drink indices
    drink_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df_new[['strDrink', 'all_ingredients']].iloc[drink_indices]

In [22]:
get_recommendations('Manhattan')

,strDrink,all_ingredients
252,Martinez 2,"Gin,Sweet Vermouth,Maraschino Liqueur,Angostur..."
277,Old Fashioned,"Bourbon,Angostura bitters,Sugar,Water"
49,Brooklyn,"Rye Whiskey,Dry Vermouth,Maraschino Liqueur,An..."
50,Boomerang,"Gin,Dry Vermouth,Bitters,Maraschino liqueur,Ma..."
35,Artillery,"Sweet Vermouth,Gin,Bitters"
65,Casino,"Gin,Maraschino liqueur,Lemon juice,Orange bitt..."
29,Affinity,"Scotch,Sweet Vermouth,Dry Vermouth,Orange bitters"
196,John Collins,"Bourbon,Lemon juice,Sugar,Club soda,Maraschino..."
377,Tuxedo Cocktail,"Dry Vermouth,Gin,Maraschino liqueur,Anis,Orang..."
287,Pegu Club,"Gin,Orange Curacao,Lime Juice,Angostura Bitter..."


In [30]:
# Prepare TF-IDF
#def prep_tfidf(drink):
df = pd.read_csv('../static/cocktaildb.csv')

# Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

# Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df['soup'])

# Output the shape of tfidf_matrix
#tfidf_matrix.shape

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Construct a reverse map of indices and movie titles
indices = pd.Series(df.index, index=df['strDrink']).drop_duplicates()

def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the drink indices
    drink_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df.iloc[drink_indices]

In [31]:
rec_list = get_recommendations('Manhattan')
rec_list

,Unnamed: 0,idDrink,strDrink,strCategory,strIBA,strAlcoholic,strGlass,strInstructions,strDrinkThumb,strIngredient1,...,strMeasure3,strMeasure4,strMeasure5,strMeasure6,strMeasure7,strMeasure8,strMeasure9,strMeasure10,all_ingredients,soup
252,252,17256,Martinez 2,Cocktail,NaN,Alcoholic,Cocktail glass,Add all ingredients to a mixing glass and fill...,https://www.thecocktaildb.com/images/media/dri...,Gin,...,1 tsp,2 dashes,NaN,NaN,NaN,NaN,NaN,NaN,"Gin,Sweet Vermouth,Maraschino Liqueur,Angostur...","Gin,Sweet Vermouth,Maraschino Liqueur,Angostur..."
49,49,178310,Brooklyn,Cocktail,NaN,Alcoholic,Cocktail glass,Combine ingredients with ice and stir until we...,https://www.thecocktaildb.com/images/media/dri...,Rye Whiskey,...,1/4 oz,3 dashes,1,NaN,NaN,NaN,NaN,NaN,"Rye Whiskey,Dry Vermouth,Maraschino Liqueur,An...","Rye Whiskey,Dry Vermouth,Maraschino Liqueur,An..."
50,50,11124,Boomerang,Ordinary Drink,NaN,Alcoholic,Cocktail glass,"In a mixing glass half-filled with ice cubes, ...",https://www.thecocktaildb.com/images/media/dri...,Gin,...,2 dashes,1/2 tsp,1,NaN,NaN,NaN,NaN,NaN,"Gin,Dry Vermouth,Bitters,Maraschino liqueur,Ma...","Gin,Dry Vermouth,Bitters,Maraschino liqueur,Ma..."
196,196,11580,John Collins,Ordinary Drink,NaN,Alcoholic,Collins glass,Pour all ingredients directly into highball gl...,https://www.thecocktaildb.com/images/media/dri...,Bourbon,...,1 tsp superfine,3 oz,1,1,NaN,NaN,NaN,NaN,"Bourbon,Lemon juice,Sugar,Club soda,Maraschino...","Bourbon,Lemon juice,Sugar,Club soda,Maraschino..."
65,65,17185,Casino,Ordinary Drink,Unforgettables,Alcoholic,Cocktail glass,Pour all ingredients into shaker with ice cube...,https://www.thecocktaildb.com/images/media/dri...,Gin,...,1/4 tsp,2 dashes,1,NaN,NaN,NaN,NaN,NaN,"Gin,Maraschino liqueur,Lemon juice,Orange bitt...","Gin,Maraschino liqueur,Lemon juice,Orange bitt..."
35,35,11055,Artillery,Ordinary Drink,NaN,Alcoholic,Cocktail glass,"Stir all ingredients with ice, strain into a c...",https://www.thecocktaildb.com/images/media/dri...,Sweet Vermouth,...,2 dashes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Sweet Vermouth,Gin,Bitters","Sweet Vermouth,Gin,Bitters,Stir all ingredient..."
373,373,12418,Turf Cocktail,Ordinary Drink,NaN,Alcoholic,Cocktail glass,Stir all ingredients (except orange peel) with...,https://www.thecocktaildb.com/images/media/dri...,Dry Vermouth,...,1/4 tsp,2 dashes,Twist of,NaN,NaN,NaN,NaN,NaN,"Dry Vermouth,Gin,Anis,Bitters,Orange peel","Dry Vermouth,Gin,Anis,Bitters,Orange peel,Stir..."
287,287,17249,Pegu Club,Cocktail,NaN,Alcoholic,Cocktail glass,"Shake, strain, up, cocktail glass",https://www.thecocktaildb.com/images/media/dri...,Gin,...,3/4 oz,1 dash,1 dash,NaN,NaN,NaN,NaN,NaN,"Gin,Orange Curacao,Lime Juice,Angostura Bitter...","Gin,Orange Curacao,Lime Juice,Angostura Bitter..."
377,377,12420,Tuxedo Cocktail,Ordinary Drink,Unforgettables,Alcoholic,Cocktail glass,Stir all ingredients with ice and strain into ...,https://www.thecocktaildb.com/images/media/dri...,Dry Vermouth,...,1/4 tsp,1/4 tsp,2 dashes,1,NaN,NaN,NaN,NaN,"Dry Vermouth,Gin,Maraschino liqueur,Anis,Orang...","Dry Vermouth,Gin,Maraschino liqueur,Anis,Orang..."
29,29,17840,Affinity,Ordinary Drink,NaN,Alcoholic,Cocktail glass,"In a mixing glass half-filled with ice cubes, ...",https://www.thecocktaildb.com/images/media/dri...,Scotch,...,1 oz,2 dashes,NaN,NaN,NaN,NaN,NaN,NaN,"Scotch,Sweet Vermouth,Dry Vermouth,Orange bitters","Scotch,Sweet Vermouth,Dry Vermouth,Orange bitt..."


In [25]:
rec_list['strDrinkThumb'].iloc[0]

'https://www.thecocktaildb.com/images/media/drink/fs6kiq1513708455.jpg'

In [26]:
rec_list['all_ingredients'].iloc[0].replace(",", ", ")

'Gin, Sweet Vermouth, Maraschino Liqueur, Angostura Bitters'

# Loading straight from CSV

In [2]:
import pandas as pd

In [11]:
df = pd.read_csv('../static/cocktaildb.csv')
df.head()

,Unnamed: 0,idDrink,strDrink,strCategory,strIBA,strAlcoholic,strGlass,strInstructions,strDrinkThumb,strIngredient1,...,strMeasure3,strMeasure4,strMeasure5,strMeasure6,strMeasure7,strMeasure8,strMeasure9,strMeasure10,all_ingredients,soup
0,0,15346,155 Belmont,Cocktail,NaN,Alcoholic,White wine glass,Blend with ice. Serve in a wine glass. Garnish...,https://www.thecocktaildb.com/images/media/dri...,Dark rum,...,1 shot,1 shot,NaN,NaN,NaN,NaN,NaN,NaN,"Dark rum, Light rum, Vodka, Orange juice","Dark rum, Light rum, Vodka, Orange juice,Blend..."
1,1,15395,1-900-FUK-MEUP,Shot,NaN,Alcoholic,Old-fashioned glass,Shake ingredients in a mixing tin filled with ...,https://www.thecocktaildb.com/images/media/dri...,Absolut Kurant,...,1/4 oz,1/4 oz,1/4 oz,1/4 oz,1/2 oz,1/4 oz,NaN,NaN,"Absolut Kurant, Grand Marnier, Chambord raspbe...","Absolut Kurant, Grand Marnier, Chambord raspbe..."
2,2,15423,110 in the shade,Beer,NaN,Alcoholic,Beer Glass,Drop shooter in glass. Fill with beer,https://www.thecocktaildb.com/images/media/dri...,Lager,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Lager, Tequila","Lager, Tequila,Drop shooter in glass. Fill wit..."
3,3,14588,151 Florida Bushwacker,Milk / Float / Shake,NaN,Alcoholic,Beer mug,Combine all ingredients. Blend until smooth. G...,https://www.thecocktaildb.com/images/media/dri...,Malibu rum,...,1/2 oz Bacardi,1 oz,1 oz,3 oz,1 oz,1 cup,NaN,NaN,"Malibu rum, Light rum, 151 proof rum, Dark Cre...","Malibu rum, Light rum, 151 proof rum, Dark Cre..."
4,4,15288,252,Shot,NaN,Alcoholic,Shot glass,"Add both ingredients to shot glass, shoot, and...",https://www.thecocktaildb.com/images/media/dri...,151 proof rum,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"151 proof rum, Wild Turkey","151 proof rum, Wild Turkey,Add both ingredient..."


In [14]:
pd.Series(' '.join(df['all_ingredients']).lower().split()).value_counts()[:50]

juice,        121
rum,           85
gin,           78
sugar,         73
orange         70
lemon          69
vodka,         63
juice          56
light          44
lime           38
vermouth,      37
lemon,         35
liqueur,       34
water,         33
cream,         29
brandy,        26
water          26
maraschino     26
powdered       26
cherry         26
bitters,       25
triple         24
sweet          24
irish          23
ice            23
sec,           23
pineapple      23
syrup,         22
grenadine,     21
peel           21
milk,          20
cream          20
lime,          20
dry            20
sugar          19
de             19
soda           19
creme          19
kahlua,        18
,              18
coffee,        17
whiskey,       17
cranberry      16
egg            16
amaretto,      16
baileys        16
tequila,       16
ginger         15
proof          15
151            15
dtype: int64

In [15]:
pd.Series(' '.join(df['all_ingredients']).lower().split()).value_counts()[50:100]

scotch,       15
white         14
vanilla       13
carbonated    13
dark          13
peach         12
raspberry     11
wine,         11
bourbon,      11
coffee        11
bitters       11
curacao,      10
angostura     10
red           10
nutmeg        10
cherry,        9
grapefruit     9
absolut        9
ice,           9
schnapps,      9
sambuca,       8
orange,        8
blended        8
extract,       8
blue           8
milk           8
soda,          8
egg,           8
grenadine      8
yoghurt,       8
apple          8
green          7
mint           7
club           7
beer,          7
grand          7
sour           7
and            7
lemonade,      7
malibu         7
galliano,      7
chambord       7
ginger,        6
marnier,       6
coca-cola,     6
ale,           6
olive          6
coca-cola      6
mint,          6
apricot        6
dtype: int64

In [16]:
pd.Series(' '.join(df['strGlass']).lower().split()).value_counts()[:50]

glass                 370
cocktail               99
highball               87
collins                60
old-fashioned          44
shot                   27
coffee                 21
mug                    21
sour                   14
whiskey                14
beer                    9
bowl                    8
punch                   8
wine                    7
champagne               7
flute                   7
hurricane               6
pitcher                 5
cup                     5
irish                   5
margarita               4
martini                 4
balloon                 3
brandy                  3
snifter                 3
and                     3
nora                    3
pint                    3
nick                    3
white                   3
cordial                 2
jar                     2
margarita/coupette      2
pilsner                 2
mason                   2
copper                  1
pousse                  1
cafe                    1
coupe       

In [45]:
df = pd.read_csv('../static/cocktaildb.csv')

a = 'vodka'
b = 'tonic'
c = 'lime'
s = [a, b, c]
ingredient_list = ' '.join([str(elem) for elem in s]) 

new_drink = {'strDrink': 'New Drink', 'soup': ingredient_list}
df = df.append(new_drink, ignore_index=True)

# Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')
# Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df['soup'])
# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
# Construct a reverse map of indices and movie titles
indices = pd.Series(df.index, index=df['strDrink']).drop_duplicates()

def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the drink indices
    drink_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df_new[['strDrink', 'soup']].iloc[drink_indices]

In [46]:
rec_list = get_recommendations('New Drink')
rec_list

,strDrink,soup
397,Vodka And Tonic,"Vodka,Tonic water,Pour vodka into a highball g..."
19,AT&T,"Absolut Vodka,Gin,Tonic water,Pour Vodka and G..."
222,Long vodka,"Vodka,Lime,Angostura bitters,Tonic water,Ice,S..."
255,Moscow Mule,"Vodka,Lime juice,Ginger ale,Combine vodka and ..."
206,Kamikaze,"Vodka,Triple sec,Lime juice,Shake all ingredie..."
13,747 Drink,"Vodka,Roses sweetened lime juice,Cranberry Jui..."
305,Queen Bee,"Coffee brandy,Lime vodka,Sherry,Shake all ingr..."
100,Darkwood Sling,"Cherry Heering,Soda water,Orange juice,Ice,The..."
332,Raspberry Cooler,"Raspberry vodka,Lemon-lime soda,Ice,Pour the r..."
327,Rosemary Blue,"Gin,Blue Curacao,Tonic Water,Rosemary,1) Add t..."
